<a href="https://colab.research.google.com/github/Lenaami/SpeechSynthesisLabs/blob/main/SS_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
path = '/content/gdrive/My Drive/Colab Notebooks/Синтез речи/'

In [3]:
book1 = 'r_hod.Result.xml'
book2 = 'tropa.Result.xml'
book3 = 'whtguard.Result.xml'

# Парсинг xml

In [4]:
import xml.etree.ElementTree as ET 
import numpy as np
import re

In [5]:
def isNone(a):
    return int(a) if a is not None else -1

In [6]:
# Создание словарей

letters = ['']
phonemes = ['']
allophones = ['']

books = [book1, book2, book3]
#books = [book1]

for book in books:
    tree = ET.parse(path + book)
    root = tree.getroot()

    for snt in root.findall('sentence'):
        for feat in snt:
            if feat.tag == 'word':         
                for lt in feat:
                    if lt.tag == 'letter': 
                        letters.append(lt.get('char'))
                    if lt.tag == 'phoneme': 
                        phonemes.append(lt.get('ph'))
                    if lt.tag == 'allophone': 
                        allophones.append(lt.get('ph'))    

letters = set(letters)
phonemes = set(phonemes)
allophones = set(allophones)

letters_dict = {lt:i for i,lt in enumerate(letters)}
phonemes_dict = {ph:i for i,ph in enumerate(phonemes)}
allophones_dict = {ph:i for i,ph in enumerate(allophones)}

num_to_letters = {i:lt for i,lt in enumerate(letters)}
num_to_phonemes = {i:ph for i,ph in enumerate(phonemes)}
num_to_allophones = {i:ph for i,ph in enumerate(allophones)}

In [7]:
def get_data(file):  
    X_phoneme = []
    X_allophone = []
    y_phoneme = []
    y_allophone = []

    tree = ET.parse(file)
    root = tree.getroot()

    tags = ['word', 'pause']

    for snt in root.findall('sentence'):
        features = []
        feat_wrd = []

        features_lt = []
        features_lt_stc = []
        letters = []
        feat_lt = []
        let_wrd = []
        ph_wrd = []
        allph_wrd = []

        ys_phoneme = []
        ys_allophone = []

        pause_pred = 0
        count = 0
        for feat in snt:

            if feat.tag not in tags:
                continue

            if feat.tag == 'word':
                if pause_pred:
                    feat_wrd.append(1) # пауза до слова
                    feat_wrd.append(0) # пауза после слова (предв.)
                    pause_pred = 0
                else:
                    feat_wrd.append(0)
                    feat_wrd.append(0)

                dct = feat.find('dictitem')
                feat_wrd.append(isNone(dct.get('stress_dict'))) # ударение
                feat_wrd.append(isNone(dct.get('genesys'))) # одушевленность
                feat_wrd.append(isNone(dct.get('form'))) # форма
                feat_wrd.append(isNone(dct.get('subpart_of_speech'))) # часть речи        
                feat_wrd.append(isNone(dct.get('semantics1')))
                feat_wrd.append(isNone(dct.get('semantics2')))
                feat_wrd.append(-1) # ударение (предв.)
                
                let_wrd = [''] # нет символа до слова
                ph_wrd = ['']
                allph_wrd = []
                features_lt = []
                gl = 0
                sogl = 0
                for lt in feat:                    
                    
                    if lt.tag == 'letter':
                        feat_lt = []
                        let_wrd.append(lt.get('char'))  
                        feat_lt.append(sogl + gl) # позиция символа в слове
                        feat_lt.append(sogl) # число согласных до
                        feat_lt.append(gl) # число гласных до
                        if lt.get('flag') == '16' or lt.get('flag') is None:  # флаги букв  (не равно 16, none - гласная)
                            sogl += 1
                            feat_lt.append(1) # согласная
                        else:
                            gl += 1
                            feat_lt.append(0) # гласная  
                        features_lt.append(feat_lt)

                        ph_wrd.append('')
                        allph_wrd.append('')

                    if lt.tag == 'phoneme':
                        ph_wrd[-1] = lt.get('ph')

                    if lt.tag == 'allophone':
                        allph_wrd[-1] = lt.get('ph')

                    if lt.tag == 'stress':
                        feat_wrd[-1] = len(let_wrd) # позиция ударной буквы                    

                let_wrd.append('') # нет символа после слова
                ph_wrd.append('')

                feat_wrd.append(sogl) # число согласных
                feat_wrd.append(gl) # число гласных

                feat_wrd.append(count) # позиция слова в предложении
                count += 1 # подсчет слов              
               
                features.append(feat_wrd)
                feat_wrd = []

                letters.append(let_wrd)
                features_lt_stc.append(features_lt)
                ys_phoneme.append(ph_wrd)
                ys_allophone.append(allph_wrd)


            if feat.tag == 'pause':
                features[-1][1] = 1 # пауза после слова
                pause_pred = 1             

        # объединение 
        for i in range(count):
            wrd_ph = []
            wrd_allph = []
            for l in range(len(letters[i]) - 2):
                feat = [letters[i][l+1], letters[i][l], letters[i][l+2]] # текущая буква, перед и после
                feat = [letters_dict.get(s) for s in feat] # перевод из буквы в цифру
                feat.extend(features_lt_stc[i][l])
                feat.extend(features[i])
                feat.append(count)
                wrd_ph.append(feat)

                feat = [ys_phoneme[i][l+1], ys_phoneme[i][l], ys_phoneme[i][l+2]] # текущая фонема, перед и после
                feat = [phonemes_dict.get(s) for s in feat]
                feat.extend(features_lt_stc[i][l])
                feat.extend(features[i])
                feat.append(count)
                wrd_allph.append(feat)
           
            ys_phoneme[i] = [phonemes_dict.get(s) for s in ys_phoneme[i][1:-1]]
            ys_allophone[i] = [allophones_dict.get(s) for s in ys_allophone[i]]
            y_allophone.extend(ys_allophone[i])
            X_phoneme.append(wrd_ph)
            X_allophone.extend(wrd_allph)

        y_phoneme.extend(ys_phoneme)
       
    return X_phoneme, X_allophone, y_phoneme, y_allophone

In [8]:
X_train_ph_1, X_train_allph_1, y_train_ph_1, y_train_allph_1 = get_data(path + book2)

In [9]:
X_train_ph_2, X_train_allph_2, y_train_ph_2, y_train_allph_2 = get_data(path + book3)

In [10]:
# Объединение корпусов (2 и 3)

X_train_ph = []
X_train_ph.extend(X_train_ph_1)
X_train_ph.extend(X_train_ph_2)

y_train_ph = []
y_train_ph.extend(y_train_ph_1)
y_train_ph.extend(y_train_ph_2)

X_train_allph = []
X_train_allph.extend(X_train_allph_1)
X_train_allph.extend(X_train_allph_2)

y_train_allph = []
y_train_allph.extend(y_train_allph_1)
y_train_allph.extend(y_train_allph_2)

In [11]:
# Тестовый корпус (1)

X_test_ph, X_test_allph, y_test_ph, y_test_allph = get_data(path + book1)

# Определение фонем

## LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from sklearn.metrics import f1_score
from tqdm import tqdm

torch.manual_seed(1)

In [142]:
class LSTMModel(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, word):
        feats = torch.tensor(data=word, dtype=torch.float)
        lstm_out, _ = self.lstm(feats.view(-1, 1, self.embedding_dim))
        tag_space = self.hidden2tag(lstm_out.view(len(word), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [202]:
def eval_model(model, eval_dataset):
    model.eval()
    forecast, true_labels = [], []
    with torch.no_grad():
        for features, labels in tqdm(eval_dataset):
            features = torch.tensor(features).cuda()
            true_labels.append(labels)
            outputs = model(features)
            
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labels = [x for sublist in true_labels for x in sublist]
    return f1_score(forecast, true_labels, average='weighted')

In [ ]:
lr = 1e-3
EMBEDDING_DIM = 20
HIDDEN_DIM = 20

model = LSTMModel(EMBEDDING_DIM, HIDDEN_DIM, len(phonemes_dict))
model = model.cuda()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

with torch.no_grad():
    inputs = torch.tensor(X_train_ph[0]).cuda()
    tag_scores = model(inputs)
    print(tag_scores)

In [23]:
lr = 1e-3
EMBEDDING_DIM = 20
HIDDEN_DIM = 200

model = LSTMModel(EMBEDDING_DIM, HIDDEN_DIM, len(phonemes_dict))
model = model.cuda()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

n_epoch = 10
for epoch in range(n_epoch):
    model.train()
    for features, labels in tqdm(zip(X_train_ph, y_train_ph)):
        optimizer.zero_grad()
        features, labels = torch.tensor(features).cuda(), torch.tensor(labels).cuda()
        outputs = model(features)
        loss = criterion(outputs, torch.tensor(labels))
        loss.backward()
        optimizer.step()
    f1 = eval_model(model, zip(X_test_ph, y_test_ph))
    f1_train = eval_model(model, zip(X_train_ph, y_train_ph))
    print(f'\nepoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
193701it [05:45, 560.65it/s]
1268it [00:00, 1886.36it/s]
193701it [01:40, 1918.11it/s]
58it [00:00, 571.40it/s]


epoch: 0, f1_test: 0.33034182025306397, f1_train: 0.35497657341642724


193701it [05:50, 553.09it/s]
1268it [00:00, 1979.66it/s]
193701it [01:44, 1858.91it/s]
58it [00:00, 577.11it/s]


epoch: 1, f1_test: 0.46146895985815495, f1_train: 0.48025782950600737


193701it [05:56, 542.90it/s]
1268it [00:00, 1934.32it/s]
193701it [01:46, 1822.87it/s]
53it [00:00, 524.57it/s]

...


epoch: 7, f1_test: 0.6867575272866396, f1_train: 0.6985769806314697


193701it [05:53, 548.63it/s]
1268it [00:00, 1872.64it/s]
193701it [01:43, 1867.74it/s]
54it [00:00, 537.72it/s]


epoch: 8, f1_test: 0.695252810408832, f1_train: 0.7137517391737785


193701it [05:54, 545.81it/s]
1268it [00:00, 1868.96it/s]
193701it [01:43, 1870.18it/s]



epoch: 9, f1_test: 0.7001625680744026, f1_train: 0.7219106102451079


In [29]:
n_epoch = 10
for epoch in range(n_epoch):
    model.train()
    for features, labels in tqdm(zip(X_train_ph, y_train_ph)):
        #print(labels)
        optimizer.zero_grad()
        features, labels = torch.tensor(features).cuda(), torch.tensor(labels).cuda()
        outputs = model(features)
        loss = criterion(outputs, torch.tensor(labels))
        loss.backward()
        optimizer.step()
    f1 = eval_model(model, zip(X_test_ph, y_test_ph))
    f1_train = eval_model(model, zip(X_train_ph, y_train_ph))
    print(f'\nepoch: {epoch + 10}, f1_test: {f1}, f1_train: {f1_train}')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
193701it [05:43, 564.00it/s]
1268it [00:00, 1948.01it/s]
193701it [01:40, 1921.01it/s]
58it [00:00, 578.35it/s]


epoch: 10, f1_test: 0.7131621325236558, f1_train: 0.7358476606036586


193701it [05:41, 568.02it/s]
1268it [00:00, 1916.34it/s]
193701it [01:40, 1929.74it/s]
56it [00:00, 554.62it/s]


epoch: 11, f1_test: 0.7163282182467872, f1_train: 0.7422957731411414


193701it [05:41, 566.59it/s]
1268it [00:00, 1940.89it/s]
193701it [01:40, 1935.87it/s]
56it [00:00, 556.92it/s]

...


epoch: 17, f1_test: 0.7556772825733393, f1_train: 0.7805590794689227


193701it [05:50, 552.63it/s]
1268it [00:00, 1865.93it/s]
193701it [01:41, 1906.32it/s]
56it [00:00, 559.77it/s]


epoch: 18, f1_test: 0.7549832026674964, f1_train: 0.783956822817986


193701it [05:44, 561.75it/s]
1268it [00:00, 1932.53it/s]
193701it [01:40, 1931.72it/s]



epoch: 19, f1_test: 0.7645419184668699, f1_train: 0.7885676858984794


In [156]:
lr = 1e-3

EMBEDDING_DIM = 20
HIDDEN_DIM = 200

model = LSTMModel(EMBEDDING_DIM, HIDDEN_DIM, len(phonemes_dict))
model = model.cuda()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

n_epoch = 20
for epoch in range(n_epoch):
    model.train()
    for features, labels in tqdm(zip(X_train_ph, y_train_ph)):
        optimizer.zero_grad()
        features, labels = torch.tensor(features).cuda(), torch.tensor(labels).cuda()
        outputs = model(features)
        loss = criterion(outputs, torch.tensor(labels))
        loss.backward()
        optimizer.step()
    f1 = eval_model(model, zip(X_test_ph, y_test_ph))
    f1_train = eval_model(model, zip(X_train_ph, y_train_ph))
    print(f'\nepoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
193701it [05:59, 538.35it/s]
1268it [00:00, 1817.82it/s]
193701it [01:45, 1842.78it/s]
56it [00:00, 556.82it/s]


epoch: 0, f1_test: 0.32485490271952433, f1_train: 0.35786701486110134


193701it [05:59, 538.36it/s]
1268it [00:00, 1750.44it/s]
193701it [01:44, 1853.67it/s]
55it [00:00, 549.63it/s]


epoch: 1, f1_test: 0.4343907635018853, f1_train: 0.45490339781534583


193701it [05:55, 544.62it/s]
1268it [00:00, 1883.89it/s]
193701it [01:45, 1842.20it/s]
46it [00:00, 452.62it/s]

...


epoch: 17, f1_test: 0.6899248585085316, f1_train: 0.7418452569874339


193701it [06:05, 530.36it/s]
1268it [00:00, 1757.09it/s]
193701it [01:47, 1807.05it/s]
55it [00:00, 548.60it/s]


epoch: 18, f1_test: 0.694913179631392, f1_train: 0.7456312555837468


193701it [06:04, 530.96it/s]
1268it [00:00, 1926.63it/s]
193701it [01:46, 1813.40it/s]



epoch: 19, f1_test: 0.702576924479888, f1_train: 0.7508214001916408


In [31]:
eval_model(model, zip(X_test_ph, y_test_ph))

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
1268it [00:00, 1841.27it/s]


0.7645419184668699

In [35]:
i = 3
with torch.no_grad():    
    outputs = model(torch.tensor(X_test_ph[i]).cuda())            
    outputs = outputs.detach().cpu().numpy().argmax(axis=1)
    print(outputs)
    
    out = [num_to_phonemes.get(s) for s in outputs]
    print(out)
    print([num_to_phonemes.get(s) for s in y_test_ph[i]])

[30 29 20 10  0 12 39 42]
['к', "р'", 'е', 'с', '', 'н', 'ы', 'й']
['к', "р'", 'е', 'с', '', 'н', 'ы', 'й']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [157]:
torch.save(model.state_dict(), path + 'model_test.pt')

In [158]:
# Загрузить модель
model_name = 'model_test.pt'

EMBEDDING_DIM = 20
HIDDEN_DIM = 200
model = LSTMModel(EMBEDDING_DIM, HIDDEN_DIM, len(phonemes_dict)).cuda()
model.load_state_dict(torch.load(path + 'model_test.pt'))
model.eval()

LSTMModel(
  (lstm): LSTM(20, 200)
  (hidden2tag): Linear(in_features=200, out_features=43, bias=True)
)

In [212]:
eval_model(model, zip(X_test_ph, y_test_ph))

1268it [00:00, 1794.83it/s]


0.845719740332514

In [ ]:
pip install python-Levenshtein

In [13]:
import Levenshtein
from sklearn import metrics

In [14]:
len_word = [0]
for y in y_test_ph:
    len_word.append(len_word[-1] + len(y))

In [15]:
def show_metric_ph(y_pred, y_true, metric='micro'):

    lev_score = []

    for i in range(len(len_word) - 1):
        pred = "".join([num_to_phonemes.get(s) for s in y_pred[len_word[i]:len_word[i+1]]])
        true = "".join([num_to_phonemes.get(s) for s in y_true[len_word[i]:len_word[i+1]]])

        lev_score.append(Levenshtein.distance(pred, true))    

    print('\nF1-score (%s):\t%0.3f' % (metric, metrics.f1_score(y_true, y_pred, average=metric)))
    print('Recall (%s): \t%0.3f' % (metric, metrics.recall_score(y_true, y_pred, average=metric)))
    print('Precision (%s):\t%0.3f' % (metric,metrics.precision_score(y_true, y_pred, average=metric)))
    print('Accuracy:\t\t%0.3f' % metrics.accuracy_score(y_true, y_pred))
    print('\nLevenshtein distance:\t%0.3f' % np.mean(lev_score))

In [16]:
def fit_clf(clf, X_train, y_train):
    clf.fit(X_train, y_train)
    return clf

def predict_clf(clf, X_test, y_test, metric='micro'):
    y_pred = clf.predict(X_test)
    show_metric_ph(y_pred, y_test, metric)
    return y_pred

In [211]:
y_pred = []
y_true = []

with torch.no_grad():
    for features, labels in tqdm(zip(X_test_ph, y_test_ph)):
        outputs = model(torch.tensor(features).cuda())            
        outputs = outputs.detach().cpu().numpy().argmax(axis=1)

        y_pred.extend(outputs)
        y_true.extend(labels)

show_metric_ph(y_pred, y_true, 'weighted') # 'weighted' - чуть ниже значения, но разные

1268it [00:00, 1690.80it/s]


F1-score (weighted):	0.837
Recall (weighted): 	0.841
Precision (weighted):	0.843
Accuracy:		0.841

Levenshtein distance:	0.943


## Классификаторы

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
X_train_ph_2 = [x for sublist in X_train_ph for x in sublist]
y_train_ph_2 = [x for sublist in y_train_ph for x in sublist]

X_test_ph_2 = [x for sublist in X_test_ph for x in sublist]
y_test_ph_2 = [x for sublist in y_test_ph for x in sublist]

In [195]:
# Decision Tree

tree_0 = DecisionTreeClassifier()

tree_0 = fit_clf(tree_0, X_train_ph_2, y_train_ph_2)
y_pred = predict_clf(tree_0, X_test_ph_2, y_test_ph_2) #'weighted'

F1-score (micro):	0.980
Recall (micro): 	0.980
Precision (micro):	0.980
Accuracy:		0.980

Levenshtein distance:	0.029


In [20]:
tree_0 = DecisionTreeClassifier()

tree_0 = fit_clf(tree_0, X_train_ph_2, y_train_ph_2)
y_pred = predict_clf(tree_0, X_test_ph_2, y_test_ph_2, 'weighted') #'weighted'


F1-score (weighted):	0.977
Recall (weighted): 	0.977
Precision (weighted):	0.977
Accuracy:		0.977

Levenshtein distance:	0.123


In [24]:
# Random Forest

rf_0 = RandomForestClassifier(n_estimators=15)

rf_0 = fit_clf(rf_0, X_train_ph_2, y_train_ph_2)
y_pred = predict_clf(rf_0, X_test_ph_2, y_test_ph_2, 'weighted') #'weighted'


F1-score (weighted):	0.959
Recall (weighted): 	0.960
Precision (weighted):	0.960
Accuracy:		0.960

Levenshtein distance:	0.222


In [22]:
# Naive Baies

nb_0 = GaussianNB(var_smoothing=1e-5) 

nb_0 = fit_clf(nb_0, X_train_ph_2, y_train_ph_2)
y_pred = predict_clf(nb_0, X_test_ph_2, y_test_ph_2, 'weighted')


F1-score (weighted):	0.469
Recall (weighted): 	0.481
Precision (weighted):	0.509
Accuracy:		0.481

Levenshtein distance:	2.811


In [23]:
# Logistic Regression

lr_0 = LogisticRegression()

lr_0 = fit_clf(lr_0, X_train_ph_2, y_train_ph_2)
y_pred = predict_clf(lr_0, X_test_ph_2, y_test_ph_2, 'weighted')


F1-score (weighted):	0.264
Recall (weighted): 	0.307
Precision (weighted):	0.246
Accuracy:		0.307

Levenshtein distance:	4.014


#Классификация фонем

In [25]:
def fit_clf(clf, X_train, y_train):
    clf.fit(X_train, y_train)
    return clf

def predict_clf(clf, X_test, y_test, metric='micro'):
    y_pred = clf.predict(X_test)
    show_metric(y_pred, y_test, metric)
    return y_pred

def show_metric(y_pred, y_true, metric):

    lev_score = []
    for i in range(len(len_word) - 1):
        pred = "".join([num_to_allophones.get(s) for s in y_pred[len_word[i]:len_word[i+1]]])
        true = "".join([num_to_allophones.get(s) for s in y_true[len_word[i]:len_word[i+1]]])
        lev_score.append(Levenshtein.distance(pred, true))    

    print('F1-score (%s):\t%0.3f' % (metric, metrics.f1_score(y_true, y_pred, average=metric)))
    print('Recall (%s): \t%0.3f' % (metric, metrics.recall_score(y_true, y_pred, average=metric)))
    print('Precision (%s):\t%0.3f' % (metric,metrics.precision_score(y_true, y_pred, average=metric)))
    print('Accuracy:\t\t%0.3f' % metrics.accuracy_score(y_true, y_pred))
    print('\nLevenshtein distance:\t%0.3f' % np.mean(lev_score))

## Decision Tree

In [28]:
# критерий разделения, максимальная глубина дерева, минимальное число объектов в листе, максимальное число листьев
#tree = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=5, max_leaf_nodes=100) 
#tree = DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=15, max_leaf_nodes=25)
tree = DecisionTreeClassifier()

tree = fit_clf(tree, X_train_allph, y_train_allph)
y_pred = predict_clf(tree, X_test_allph, y_test_allph, 'weighted') #'weighted'

F1-score (weighted):	0.992
Recall (weighted): 	0.992
Precision (weighted):	0.991
Accuracy:		0.992

Levenshtein distance:	0.041


## Random Forest

In [29]:
rf = RandomForestClassifier(n_estimators=15)

rf = fit_clf(rf, X_train_allph, y_train_allph)
y_pred = predict_clf(rf, X_test_allph, y_test_allph, 'weighted')

F1-score (weighted):	0.973
Recall (weighted): 	0.974
Precision (weighted):	0.973
Accuracy:		0.974

Levenshtein distance:	0.179


## Naive Baies 



In [30]:
nb = GaussianNB(var_smoothing=1e-5) 

nb = fit_clf(nb, X_train_allph, y_train_allph)
y_pred = predict_clf(nb, X_test_allph, y_test_allph, 'weighted')

F1-score (weighted):	0.862
Recall (weighted): 	0.858
Precision (weighted):	0.871
Accuracy:		0.858

Levenshtein distance:	0.756


## Logisric Regression


In [31]:
lr = LogisticRegression()

lr = fit_clf(lr, X_train_allph, y_train_allph)
y_pred = predict_clf(lr, X_test_allph, y_test_allph, 'weighted')

F1-score (weighted):	0.303
Recall (weighted): 	0.338
Precision (weighted):	0.300
Accuracy:		0.338

Levenshtein distance:	5.032
